# Schrödinger's Equation

In [ ]:
# Helper code

import sys
if 'pyodide' in sys.modules:
    %pip install ipywidgets
    %pip install ipympl

import numpy             as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

%run "animate.ipy"

plt.rc('figure', figsize=(4,3))

#save_path = "../data/schrodinger"
#load_path = save_path
load_path = "https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/schrodinger"

ANIM = False
MKVIDEO = not ANIM and True
SAVEVIDEO = False
PREVIEW = not ANIM and not MKVIDEO

if ANIM:
    %matplotlib widget
else:
    %matplotlib inline

In [ ]:
def shxt(w,cm='bwr',vsym=True):
    """show temporal evolution of wave"""
    if vsym:
        r = np.abs(w).max()
        plt.imshow(np.flipud(w),cm,extent=(x1,x2,t1,t2),vmin=-r,vmax=r)
    else:
        plt.imshow(np.flipud(w),cm,extent=(x1,x2,t1,t2))
    plt.xticks(np.arange(x1,x2+1,1)); plt.yticks(np.arange(t1,t2+1,1)); plt.grid();
    plt.xlabel('x'); plt.ylabel('t');

In [ ]:
def makeAnimationArray( psi, x, nframes=100, init=None, video=True):

    fig, ax = plt.subplots(figsize=(10,3))
    plt.tight_layout(h_pad=0,w_pad=0)
    ax.set_xlim(( x[0], x[-1]))
    ax.set_ylim(-0.05, 2)

    SCA = 1.9 / (np.abs(psi[0])**2).max()
    SC = np.sqrt(SCA)

    if video:
        plt.close()

    line1, = ax.plot([], [], 'black',lw=2)
    line2, = ax.plot([],[],'blue',alpha=0.5)
    line3, = ax.plot([],[],'red',alpha=0.5)
    #line2, = ax.plot(x, V, 'gray')
    #info = ax.text(x1+0.2,2-0.2,'')

    def fotogram(i):
        r = psi[i]
        line2.set_data(x,SC*np.real(r)/2+1)
        line3.set_data(x,SC*np.imag(r)/2+1)
        line1.set_data(x,SCA*abs(r)**2)
        #info.set_text(f'{i} norm={np.sum(np.abs(r**2)):.3f}')
        return ()

    if init is not None:
        fotogram(init)

    return metaAnimation(fig,fotogram,nframes,video)

In [ ]:
def makeAnimation( psi, x, nframes=100, init=None, video=True):

    fig, ax = plt.subplots(figsize=(10,3))
    plt.tight_layout(h_pad=0,w_pad=0)
    ax.set_xlim(( x[0], x[-1]))
    ax.set_ylim(-0.05, 2)

    SCA = 1.9 / (np.abs(psi(0))**2).max()
    SC = np.sqrt(SCA)

    if video:
        plt.close()

    line1, = ax.plot([], [], 'black',lw=2)
    line2, = ax.plot([],[],'blue',alpha=0.5)
    line3, = ax.plot([],[],'red',alpha=0.5)
    #line2, = ax.plot(x, V, 'gray')
    #info = ax.text(x1+0.2,2-0.2,'')

    def fotogram(i):
        r = psi(i)
        line2.set_data(x,SC*np.real(r)/2+1)
        line3.set_data(x,SC*np.imag(r)/2+1)
        line1.set_data(x,SCA*abs(r)**2)
        #info.set_text(f'{i} norm={np.sum(np.abs(r**2)):.3f}')
        return ()

    if init is not None:
        fotogram(init)

    return metaAnimation(fig,fotogram,nframes,video)

## Motivation

En el notebook [exponential](exponential.ipynb) hemos visto que una onda pura que se desplaza con velocidad de fase $v$ se factoriza en dos exponenciales complejas, una con frecuencia espacial $k$ y la otra con frecuencia temporal $\nu = k v$.

$$f(x,t) = \exp[i2\pi k (x-vt)] = \exp(i2\pi k x)\; \exp(-i2\pi \nu t )$$

In [ ]:
x1, x2 = -2, 7
t1, t2 = 0, 3
dx = dt = 0.01
x = np.arange(x1,x2+dx,dx).reshape(1,-1)
t = np.arange(t1,t2+dt,dt).reshape(-1,1)

In [ ]:
def Nor(f):
    k = sum(abs(f)**2)
    return f/np.sqrt(k)

In [ ]:
def wave(k,x,f,t):
    return np.exp(2j*np.pi*(k*x-f*t))

Por ejemplo, la siguiente onda tiene $k=1$ y $\nu=2$. La velocidad de fase $v=2$.

In [ ]:
onda = wave(1, x, 2, t)

plt.figure(figsize=(8,4))
shxt(np.real(onda))

In [ ]:
metadisplay('demo1', makeAnimation, dict(psi=lambda i: onda[i], x=x.flatten()), vframes=100)

## Einstein - de Broglie

El experimento de [Davisson-Germer](https://en.wikipedia.org/wiki/Davisson%E2%80%93Germer_experiment) y el efecto fotoeléctrico conducen a las ondas de materia, que se caracterizan por tener la siguiente correspondencia entre las propiedades ondulatorias y corpusculares:

$$p = k h$$
$$E =\nu h$$

En una partícula la energía y el momento no son independientes, por tanto en una onda de materia las dos frecuencias están enlazadas. La velocidad de fase queda determinada cuando se fija una de las dos.

En una partícula libre la energía es solo cinética, por lo que

$$E = \frac{p^2}{2m}$$

In [ ]:
h = 1
hbar = h/2/np.pi
m = 1

def deBroglie(p,m,x,t):
    E = p**2/(2*m)
    f = E*h
    k = p*h
    return wave(k,x,f,t)

Como ejemplo, mostramos las ondas de materia de partículas con dos valores concretos de su momento:

In [ ]:
onda = deBroglie(2, m, x, t)

plt.figure(figsize=(8,4))
shxt(np.imag(onda))

In [ ]:
onda = deBroglie(1, m, x, t)

plt.figure(figsize=(8,4))
shxt(np.imag(onda))

La onda se extiende infinitamente en el espacio y no permite modelar un objeto localizado.

## Wave packet

Para representar o describir una partícula tenemos que recurrir a un paquete de ondas.

In [ ]:
def gaussian(sigma, x):
    return np.exp(-0.5*(x/sigma)**2)/np.sqrt(2*np.pi)/sigma

sigma = 1

dk = 0.05

km = 2

ks = np.arange(km-4/sigma,km+4/sigma+dk,dk)

onda = np.sum([deBroglie(k,1,x,t) * gaussian(1/sigma, k-km) for k in ks],axis=0)

plt.figure(figsize=(8,4))
shxt(np.imag(onda))

In [ ]:
metadisplay('demo2', makeAnimation, dict(psi=lambda i: onda[i], x=x.flatten()), vframes=200)

Lo primero que observamos es que el paquete se dispersa, debido a que cada componente se mueve a diferente velocidad. Las frecuencias temporal y espacial obedecen la relación energía-momento y no tienen un ratio constante. La velocidad de la partícula no es la velocidad de fase de la onda sino la la velocidad de grupo. 

Además, sabemos que en cualquier onda la dispersión espacial y frecuencial están relacionadas por $\sigma_x \sigma_k = 1$, lo que en un paquete de ondas de materia se traduce en $\sigma_x \sigma_p = h$ (en el caso mejor). Aparece una de las manifestaciones del principio de incertidumbre, derivado simplemente del aspecto ondulatorio de la partícula. Para definir con precisión la posición hace falta un intervalo muy amplio de longitudes de onda / momentos, y viceversa. Estas diferentes velocidades producen la dispersión de la partícula. 

Cuando se efectúa una observación de la partícula se obtiene un valor concreto de la magnitud de interés, $x$ o $p$, que será alguno de los componentes puros que mediante superposición configuran el paquete de ondas. La naturaleza hace una elección aleatoria con una probabilidad proporcional a la amplitud de la onda al cuadrado.

## Propagator

La evolución temporal del paquete de ondas anterior se ha "implementado" de forma directa, superponiendo explícitamente la evolución de los "infinitos" componente puros del paquete. Al discretizar la síntesis del paquete el procedimiento computacional no es demasiado costoso, pero en cualquier caso sería muy interesante obtener una ley dinámica que hiciera evolucionar directamente la onda, de forma análoga a las leyes de evolución de un sistema físico clásico.

En la onda viajera de una partícula libre tenemos las relaciones

$$\frac{\partial}{\partial x} f(x,t) = i 2\pi k\; f(x,t) = i \frac{p}{\hbar} f(x,t)$$

$$\frac{\partial}{\partial t} f(x,t) = -i 2\pi \nu\; f(x,t) = -i \frac{E}{\hbar} f(x,t)$$

Definimos los operadores

$$\mathbb P = -i \hbar \frac{\partial}{\partial x} $$

$$\mathbb E = i  \hbar \frac{\partial}{\partial t} $$

de modo que "extraen" de la onda el momento y la energía de la partícula:

$$\mathbb P f = p f$$

$$\mathbb E f = E f $$

Si una partícula libre cumple la relación

$$E = \frac{p^2}{2m}$$

también deben cumplirla los operadores:

$$ i  \hbar \frac{\partial}{\partial t} f  = \frac{-\hbar^2}{2m} \frac{\partial^2}{\partial x^2} f $$

Por linealidad, esto no solo lo cumplirá una onda pura, sino cualquier superposición de ellas, sin necesidad de resolver explícitamente los componentes.

Se trata de una ecuación diferencial en derivadas parciales, pero si discretizamos el dominio tenemos realmente una ecuación diferencial vectorial de primer orden del tipo

$$ \frac{df}{dt} = \frac{-i \hbar}{2m} L f $$

donde L es la matriz que aplica la aproximación discreta del Laplaciano (derivada segunda) a la función de onda discretizada.

Las ecuaciones del tipo

$$ \frac{df}{dt} = a f(t) $$

tienen como solución (volvemos a las propiedades de la exponencial):

$$f(t) =  \exp(a t) f(0) $$

y esto funciona exactamente igual con escalares que con vectores y transformaciones lineales:

$$ \dot{ \vec y}(t) = A \vec y(t) $$

$$ \vec y(t)=e^{At}\vec y(0) $$

en cuyo caso la exponencial de la matriz se puede calcular mediante un desarrollo en serie o mejor por diagonalización.

Si queremos avanzar un intervalo pequeño de tiempo $\Delta t$ construimos un "propagador"

$$U = \exp(A\Delta t)$$

con el que hacemos avanzar la solución a sucesivos valores de tiempo posteriores:

$$\vec y(t+\Delta t) = U\, \vec y(t) $$

Veamos el aspecto de los operadores diferenciales discretizados.

In [ ]:
def mkLaplacian(x,periodic=True):
    n = len(x)
    dx = x[1]-x[0]
    Lap = np.diag(-2*np.ones(n)) + np.diag(np.ones(n-1),1) + np.diag(np.ones(n-1),-1)
    if periodic:
        Lap[0,-1]=1
        Lap[-1,0]=1
    return Lap/dx**2

def mkDeriv(x,periodic=True):
    n = len(x)
    dx = x[1]-x[0]
    oper = np.diag(np.ones(n-1),1) - np.diag(np.ones(n-1),-1)
    if periodic:
        oper[0,-1] = -1
        oper[-1,0] = 1
    else:
        oper[0,[0,1]] = [-2, 2]
        oper[-1,[-2,-1]] = [-2,2]
    return oper/(2*dx)

In [ ]:
n = 10
x1 = -2
x2 =  2
x = np.linspace(x1,x2,n)
dx = x[1]-x[0]
Lap = mkLaplacian(x)
Der = mkDeriv(x)

In [ ]:
mx = np.abs(Der).max()
plt.imshow(Der[:10,:10],'bwr',vmin=-mx,vmax=mx); plt.colorbar();

In [ ]:
mx = np.abs(Lap).max()
plt.imshow(Lap[:10,:10],'bwr',vmin=-mx,vmax=mx); plt.colorbar();

In [ ]:
Lap2 = Der @ Der
mx = np.abs(Lap2).max()
plt.imshow(Lap2[:10,:10],'bwr',vmin=-mx,vmax=mx); plt.colorbar();

In [ ]:
Der @ x

In [ ]:
Lap @ x**2

In [ ]:
Der @ Der @ x**2

Funciona bien excepto en los bordes del dominio porque las funciones de prueba son son periódicas.

Construimos el laplaciano del dominio anterior.

In [ ]:
x1, x2 = -2, 7
dx = 0.01
x = np.arange(x1,x2+dx,dx)
Lap = mkLaplacian(x)

Podemos reconstruir el estado inicial o coger simplemente la primera fila del diagram x-t anterior.

In [ ]:
state0 = onda[0] # = np.sum([deBroglie(k,1,x,0) * gaussian(1/sigma, k-km) for k in ks],axis=0)

Construimos el propagador:

In [ ]:
def expma(m,a):
    l,v = np.linalg.eigh(m)
    return (v * np.exp(a*l)) @ v.conj().T

In [ ]:
H = - (hbar**2/(2*m))*Lap
prop = expma(H,-1j/hbar*dt)

Y lo aplicamos sucesivas veces al estado inicial, comparando con la evolución analítica anterior:

In [ ]:
r = [state0]
for _ in range(len(t)-1):
    r.append(prop@r[-1])

plt.figure(figsize=(8,4))
shxt(np.imag(onda)); plt.show()
plt.figure(figsize=(8,4))
shxt(np.imag(r)); plt.show()

Se observa el problema del dominio finito periódico que nos impone la discretización. La onda se sale por la derecha y entra por la izquierda, produciendo una interferencia que no existe en la evolución analítica. Debido a esta limitación en las simulaciones posteriores debemos establecer dominios suficientemente amplios para mostrar la evolución de la onda antes de que se cierre sobre sí misma.

## Schrödinger's Equation

En resumen, una partícula libre tiene una onda de materia que obedece esta ley de evolución:

$$ i  \hbar \frac{\partial}{\partial t} f  = \frac{-\hbar^2}{2m} \frac{\partial^2}{\partial x^2} f $$

La derivada temporal se puede obtener de la espacial. Podría interpretarse que, instantáneamente, descompone la entrada en ondas simples, a cada una le añade su velocidad, las mueve y recompone. El efecto es equivalente pero la "implementación" es en principio mucho más directa y simple. Solo en principio, porque si la onda inicial se descompone en un número pequeño de ondas puras puede ser más eficiente trabajar expresamente con ellas.

La ecuación anterior se ha construido para que sea consistente con la dinámica de una partícula libre. Nos gustaría extenderla al caso de una partícula sometida a fuerzas que se derivan de un potencial $V(x)$. La ecuación deber seguir siendo lineal para explicar los fenómenos de interferencia, y debe ser consistente con la conservación de la energía $E=T+V=\frac{p^2}{2m} + V$, así que postulamos la siguiente ecuación, donde el operador correspondiente al potencial es él mismo:

$$ i  \hbar \frac{\partial}{\partial t} f(x,t)  = \frac{-\hbar^2}{2m} \frac{\partial^2}{\partial x^2} f(x,t) + V(x) f(x,t)$$

Se puede resolver igual que antes con un propagador. Como ejemplo, vamos a resolver el caso de una partícula en una caja, modelado con dos barreras de potencial mucho más altas que la energía de la partícula.

In [ ]:
x1, x2 = -2, 4
dx = 0.01
x = np.arange(x1,x2+dx,dx)
Lap = mkLaplacian(x)
Der = mkDeriv(x)

Como hemos visto, la envolvente gaussiana es exactamente equivalente a la ponderación gaussiana de frecuencias. Pero cuidado que por la definición de las ondas, ahora tenemos $2\pi \sigma_k \sigma_x = 1$.

In [ ]:
sigma = 1/6
km = 2

dk = 0.05
ks = np.arange(km-4/sigma,km+4/sigma+dk,dk)
state0p = np.sum([deBroglie(k,1,x-0,0) * gaussian(1/(sigma*2*np.pi), k-km) for k in ks],axis=0)

state0 = deBroglie(km,1,x,0) * gaussian(sigma,x-0)

In [ ]:
def Nor(f):
    k = sum(abs(f)**2)
    return f/np.sqrt(k)

plt.plot(x,np.real(Nor(state0p)),lw=5,alpha=0.5)
plt.plot(x,np.real(Nor(state0)),lw=1,color='black');

In [ ]:
def avg(oper,fun):
    nor = np.conj(fun) @ fun
    fun /= np.sqrt(abs(nor))
    return np.real(np.conj(fun) @ oper @ fun)

p_op = -1j*hbar*Der
x_op = np.diag(x)
Id = np.eye(len(x))

In [ ]:
avg(x_op, state0)

In [ ]:
avg(p_op, state0)

In [ ]:
Emed = avg(p_op@p_op/2/m, state0)
Emed

Comprobamos que $\left<p\right>$ es el que hemos puesto a través de `km` y que $\left<E\right> = \left< p^2 \right> /2m \neq \left<p\right>^2/2m$ 

In [ ]:
V = 20*((x>3)+(x<-1))

plt.plot(x,V);
plt.plot(x,V*0 + Emed, color='red');

In [ ]:
dt = 0.01

H = np.diag(V) - (hbar**2/(2*m))*Lap
prop = expma(H,-1j/hbar*dt)

In [ ]:
r = [state0]
for _ in range(len(t)-1):
    r.append(prop@r[-1])

plt.figure(figsize=(8,4))
shxt(np.imag(r)); plt.show()

In [ ]:
metadisplay('demo3', makeAnimation, dict(psi=lambda i: r[i], x=x), vframes=300)

## Eigenfunctions

Una forma más interesante de resolver la Ecuación de Schrödinger es mediante separación de variables. Esto da lugar a soluciones de la forma

$$\varphi_k(x)e^{-2i\pi\frac{E_k}{h}}$$

donde $\varphi_k(x)$ y $E_k$ son las soluciones a la E. de S. independiente del tiempo

$$ \left[\frac{-\hbar^2}{2m} \frac{\partial^2}{\partial x^2} + V(x)\right] \varphi(x) = H \varphi(x) = E \varphi(x)$$

esto es, las eigenfunctions del Hamiltoniano con sus respectivos autovalores. En sistemas ligados estas soluciones forman un conjunto discreto: la energía está cuantizada.

De forma análoga al caso de la partícula libre, el estado inicial se puede expandir en la base de eigenfunctions y entonces la evolución temporal se reduce a avanzar en el tiempo dichas eigenfunctions y combinarlas con los coeficientes de la expansión.

Esto es equivalente a diagonalizar el propagador:

$$H = V - \frac{\hbar^2}{2m}\nabla^2$$

$$\Psi(t) = \exp\left(\frac{t}{i\hbar} H \right)\, \Psi(0)= \exp\left(\frac{t}{i\hbar} \Phi\, E \Phi^T \right)\, \Psi(0) = \Phi\, \underbrace{\exp\left(\frac{E t}{i\hbar} \right)}_{\textrm{diagonal}} \underbrace{\Phi^T \, \Psi(0)}_{\alpha_k} $$

Podemos recortar $\Phi$ descartando las eigenfunctions que no contribuyan al estado.

In [ ]:
class schrodinger():
    def __init__(self,x1,x2,dx=None,n=None,mass=1):
        if dx is not None:
            self.x = x = np.arange(x1,x2+dx,dx)
        if n is not None:
            self.x = x = np.linspace(x1,x2,n)
        self.Lap = mkLaplacian(x)
        self.DerX = mkDeriv(x)
        self.mass = mass

    def set_potential(self,Vfun):
        self.V = V = Vfun(self.x)
        self.H = H = np.diag(V) - (hbar**2/(2*self.mass))*self.Lap
        self.E0, self.phi0 = np.linalg.eigh(H)

    def show_eigenfunctions(self,n,sc=0.1):
        plt.fill_between(self.x,0,self.V,color='lightgray')
        for k in range(n):
            plt.plot(self.x, self.E0[k]+0*self.x,color='gray',lw=1,ls='dotted')
            #plt.plot(self.x, self.E0[k]+20*sc*(self.phi[:,k]**2),color='lightgreen',lw=1)
            plt.plot(self.x, self.E0[k]+sc*(self.phi0[:,k]**1),color='black')
        plt.ylim(0,1.25*self.E0[n-1])
        plt.xlim(self.x[0],self.x[-1])
        plt.xlabel('x'); plt.ylabel('E');

    def set_initial_state(self, state, take):
        self.state0 = state
        self.phi = self.phi0[:,:take]
        self.E = self.E0[:take]
        self.coeffs = self.phi.T @ state
        self.recons = self.phi @ self.coeffs;

    def show_components(self):
        plt.plot(self.E,np.abs(self.coeffs),'.-');

    def show_initial_model(self):
        plt.plot(self.x, np.real(self.state0), lw=5, alpha=0.5)
        plt.plot(self.x, np.real(self.recons), lw=1, color='black');

    def psi(self,t):
        return (self.phi * np.exp(-1j*self.E/hbar*t)) @ self.coeffs

## Particle in a box

In [ ]:
system = schrodinger(-2,4,n=500)
system.set_potential(lambda x: 20*((x>3)+(x<-1)))

plt.figure(figsize=(4,6))
system.show_eigenfunctions(7)

In [ ]:
state0 = deBroglie(2,1,system.x,0) * gaussian(1/6, system.x-0)

system.set_initial_state(state0,45)
system.show_initial_model(); plt.show();
system.show_components()

In [ ]:
r = [system.psi(t) for t in np.arange(0,3+dt,dt)]

plt.figure(figsize=(8,5))
shxt(np.imag(r)); plt.show();
plt.figure(figsize=(8,5))
shxt(1-np.abs(r),cm='gray',vsym=False); plt.show();

In [ ]:
metadisplay('demo4',  makeAnimation, dict(psi=lambda i: r[i], x=system.x), vframes=300)

## Harmonic oscillator

In [ ]:
system = schrodinger(-5,5,n=600)
system.set_potential(lambda x: 0.5*x**2)

plt.figure(figsize=(4,6))
system.show_eigenfunctions(5,sc=0.2)

In [ ]:
onda = deBroglie(3,1,system.x,0) * gaussian(1/3, system.x-0)

In [ ]:
system.set_initial_state(onda,70)
system.show_initial_model(); plt.show();
system.show_components()

In [ ]:
dt = 0.05

r = [system.psi(t) for t in np.arange(0,2*np.pi+dt,dt)]

plt.figure(figsize=(8,5))
shxt(np.imag(r)); plt.show();
plt.figure(figsize=(8,5))
shxt(1-np.abs(r),cm='gray',vsym=False); plt.show();

In [ ]:
metadisplay('demo5',  makeAnimation, dict(psi=lambda i: r[i], x=system.x), vframes=len(r))

## Tunneling

In [ ]:
system = schrodinger(-5,7,n=600)
system.set_potential(lambda x: 20*(x<-4) + 20*(x>6) + 1*(x>3)*(x<3.2))

plt.figure(figsize=(4,6))
system.show_eigenfunctions(30,sc=0.02)

In [ ]:
onda = deBroglie(0.9,1,system.x,0) * gaussian(1/3, system.x-0)

In [ ]:
system.set_initial_state(onda,50)
system.show_initial_model(); plt.show();
system.show_components()

In [ ]:
dt = 0.05

r = [system.psi(t) for t in np.arange(0,2*np.pi+dt,dt)]

plt.figure(figsize=(8,5))
shxt(np.imag(r)); plt.show();
plt.figure(figsize=(8,5))
shxt(1-np.abs(r),cm='gray',vsym=False); plt.show();

In [ ]:
metadisplay('demo6',  makeAnimation, dict(psi=lambda i: system.psi(i*0.05), x=system.x), vframes=len(r))